In [16]:
import pandas as pd
import requests


In [17]:
deputies = pd.read_csv("data/deputados.csv")
dep_ids = list(deputies.id_dep)

In [18]:
url = "https://dadosabertos.camara.leg.br/api/v2/deputados?{}&idLegislatura=55&itens={}"
headers = {'accept': 'application/json'}

In [19]:
def get_deputies_detail(dep_ids, id_name,bulk,to_log):
    all_response = []
    no_answer = set()
    for i in range(0, len(dep_ids), bulk):
        if to_log:
            print(i, len(dep_ids))
        selected_dep_ids = dep_ids[i:i+bulk]
        ids_string = "&".join(["{}={}".format(id_name, id) for id in selected_dep_ids])
        r = requests.get(url.format(ids_string, bulk), headers=headers)
        response = r.json()
        all_response += response['dados']
        if len(response['dados']) != len(selected_dep_ids):
            no_answer |= (set(selected_dep_ids) - set([r['id'] for r in response['dados']]))
    return all_response, list(no_answer)

def get_by_name(names):
    all_response = []
    no_answer = set()
    for cur_name in names:
        ids_string = "nome={}".format(cur_name)
        r = requests.get(url.format(ids_string, bulk), headers=headers)
        response = r.json()
        all_response += response['dados']
        if len(response['dados']) == 0:
            no_answer.add(cur_name)
    return all_response, list(no_answer)

In [20]:
all_response, no_answer = get_deputies_detail(dep_ids, "id", 20, False)

In [21]:
no_answer

[]

In [22]:
names = list(deputies[deputies.id_dep.isin(no_answer)].nome)
assert len(names) == len(no_answer)

In [23]:
all_response_names, no_answer_names = get_by_name(names)

In [24]:
assert len(no_answer_names) == 0
assert len(all_response + all_response_names) == len(dep_ids)

In [25]:
pd.DataFrame(all_response + all_response_names).to_csv("data/deputados_info_by_api.csv",index=False, encoding="utf8")

In [26]:
pd.read_csv("data/deputados_info_by_api.csv")

,id,idLegislatura,nome,siglaPartido,siglaUf,uri,uriPartido,urlFoto
0,178957,55,ABEL MESQUITA JR.,DEM,RR,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
1,73579,55,ALBERTO FRAGA,DEM,DF,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
2,160545,55,ALEXANDRE LEITE,DEM,SP,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
3,74651,55,CARLOS MELLES,DEM,MG,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
4,74537,55,CLAUDIO CAJADO,DEM,BA,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
5,141422,55,EFRAIM FILHO,DEM,PB,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
6,160532,55,ELI CORRÊA FILHO,DEM,SP,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
7,178854,55,ELMAR NASCIMENTO,DEM,BA,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
8,141429,55,FELIPE MAIA,DEM,RN,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
9,160677,55,FRANCISCO FLORIANO,DEM,RJ,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
